# 1. Preparación

In [ ]:
# Instanciar y guardar el modelo

import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.applications import InceptionV3

model = InceptionV3(weights='imagenet')

model.name = "googlenet"
model.save('googlenet_model.keras', save_format='keras')



# 2. Funciones Reutilizables

In [ ]:
from load_imagenet_v2_test import load_imagenet_v2_test

# InceptionV3 requiere imágenes de tamaño 299x299
x_test, y_test = load_imagenet_v2_test(n=1000, target_size=(299, 299))


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imagenet_v2/matched-frequency/incomplete.RMIYHM_3.0.0/imagenet_v2-test.tfr…

Dataset imagenet_v2 downloaded and prepared to /root/tensorflow_datasets/imagenet_v2/matched-frequency/3.0.0. Subsequent calls will reuse this data.
✅ Dataset cargado: 1000 muestras de 299x299


In [ ]:
from exportar_tflite_variantes import exportar_tflite_variantes

# llamada a la función

exportar_tflite_variantes(model, x_test, nombre_base="googlenet")


Saved artifact at '/tmp/tmp133kg9a9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='keras_tensor_313')
Output Type:
  TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)
Captures:
  137047849616208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849618896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849618128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849618704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849618320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849619856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849617744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849616976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849619472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849620624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1370478

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


✅ Exportado: googlenet_int8.tflite
Saved artifact at '/tmp/tmp2hj398kz'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='keras_tensor_313')
Output Type:
  TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)
Captures:
  137047849616208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849618896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849618128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849618704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849618320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849619856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849617744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849616976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849619472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137047849620624: TensorSpec(shape=(), dtyp

In [ ]:
# Invocación de la función preprocess_input

from preprocess_input_tflite import preprocess_input_tflite

x_test_ready = preprocess_input_tflite("googlenet", x_test)

In [ ]:
# Invocación de la función evaluate_model

from evaluate_model import evaluate_model

acc_fp32, preds_fp32 = evaluate_model("fp32", "googlenet_fp32.tflite", x_test_ready, y_test, quantized=False)
acc_ch, preds_ch = evaluate_model("ch", "googlenet_ch.tflite", x_test_ready, y_test, quantized=False)
acc_int8, preds_int8 = evaluate_model("int8", "googlenet_int8.tflite", x_test_ready, y_test, quantized=True)



/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


✅ Modelo: FP32  → 🎯 Accuracy: 0.6070
✅ Modelo: CH  → 🎯 Accuracy: 0.6060
✅ Modelo: INT8  → 🎯 Accuracy: 0.5960


# 3. Inspección

In [ ]:
from calcular_muestreo_estadistico import calcular_muestreo_estadistico
import inspeccionar_tensores_inyectables

# Invocación FP32
df_fp32 = inspeccionar_tensores_inyectables.inspeccionar_tensores_inyectables("googlenet_fp32.tflite", tipo="fp32")

# ✅ Mostrar los tensores inyectables
print(f"✅ Tensores inyectables detectados: {len(df_fp32)}")
display(df_fp32)

✅ Tensores inyectables detectados: 188


,IndexModelo,Nombre,TipoDato,Forma,# Pesos,Bits por Peso,Total Bits Inyectables,n (muestra estadística)
0,1,arith.constant,<class 'numpy.float32'>,[384],384,32,12288,385
1,2,arith.constant1,<class 'numpy.float32'>,[384],384,32,12288,385
2,3,arith.constant2,<class 'numpy.float32'>,[384],384,32,12288,385
3,4,arith.constant3,<class 'numpy.float32'>,[448],448,32,14336,385
4,5,arith.constant4,<class 'numpy.float32'>,[384],384,32,12288,385
...,...,...,...,...,...,...,...,...
183,184,arith.constant183,<class 'numpy.float32'>,"[80, 1, 1, 64]",5120,32,163840,385
184,185,arith.constant184,<class 'numpy.float32'>,"[64, 3, 3, 32]",18432,32,589824,385
185,186,arith.constant185,<class 'numpy.float32'>,"[32, 3, 3, 32]",9216,32,294912,385
186,187,arith.constant186,<class 'numpy.float32'>,"[1000, 2048]",2048000,32,65536000,385


In [ ]:
from calcular_muestreo_estadistico import calcular_muestreo_estadistico
import inspeccionar_tensores_inyectables

# Invocación CH
df_ch = inspeccionar_tensores_inyectables.inspeccionar_tensores_inyectables("googlenet_ch.tflite", tipo="ch")

# ✅ Mostrar los tensores inyectables
print(f"✅ Tensores inyectables detectados: {len(df_ch)}")
display(df_ch)

✅ Tensores inyectables detectados: 190


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


,IndexModelo,Nombre,TipoDato,Forma,# Pesos,Bits por Peso,Total Bits Inyectables,n (muestra estadística)
0,1,arith.constant,<class 'numpy.float32'>,[32],32,32,1024,385
1,2,arith.constant1,<class 'numpy.float32'>,"[32, 3, 3, 3]",864,32,27648,385
2,4,arith.constant3,<class 'numpy.float32'>,[1000],1000,32,32000,385
3,5,arith.constant4,<class 'numpy.float32'>,[384],384,32,12288,385
4,6,arith.constant5,<class 'numpy.float32'>,[384],384,32,12288,385
...,...,...,...,...,...,...,...,...
185,187,tfl.pseudo_qconst89,<class 'numpy.int8'>,"[192, 3, 3, 80]",138240,8,1105920,385
186,188,tfl.pseudo_qconst90,<class 'numpy.int8'>,"[80, 1, 1, 64]",5120,8,40960,385
187,189,tfl.pseudo_qconst91,<class 'numpy.int8'>,"[64, 3, 3, 32]",18432,8,147456,385
188,190,tfl.pseudo_qconst92,<class 'numpy.int8'>,"[32, 3, 3, 32]",9216,8,73728,385


In [ ]:
from calcular_muestreo_estadistico import calcular_muestreo_estadistico
import inspeccionar_tensores_inyectables

# Invocación INT8
df_int8 = inspeccionar_tensores_inyectables.inspeccionar_tensores_inyectables("googlenet_int8.tflite", tipo="int8")

# ✅ Mostrar los tensores inyectables
print(f"✅ Tensores inyectables detectados: {len(df_int8)}")
display(df_int8)

✅ Tensores inyectables detectados: 95


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


,IndexModelo,Nombre,TipoDato,Forma,# Pesos,Bits por Peso,Total Bits Inyectables,n (muestra estadística)
0,3,tfl.pseudo_qconst1,<class 'numpy.int8'>,"[1000, 2048]",2048000,8,16384000,385
1,5,tfl.pseudo_qconst3,<class 'numpy.int8'>,"[384, 3, 1, 384]",442368,8,3538944,385
2,7,tfl.pseudo_qconst5,<class 'numpy.int8'>,"[384, 1, 3, 384]",442368,8,3538944,385
3,9,tfl.pseudo_qconst7,<class 'numpy.int8'>,"[384, 3, 3, 448]",1548288,8,12386304,385
4,11,tfl.pseudo_qconst9,<class 'numpy.int8'>,"[448, 1, 1, 2048]",917504,8,7340032,385
...,...,...,...,...,...,...,...,...
90,183,tfl.pseudo_qconst181,<class 'numpy.int8'>,"[192, 3, 3, 80]",138240,8,1105920,385
91,185,tfl.pseudo_qconst183,<class 'numpy.int8'>,"[80, 1, 1, 64]",5120,8,40960,385
92,187,tfl.pseudo_qconst185,<class 'numpy.int8'>,"[64, 3, 3, 32]",18432,8,147456,385
93,189,tfl.pseudo_qconst187,<class 'numpy.int8'>,"[32, 3, 3, 32]",9216,8,73728,385


# 4. Inyección

# INT 8


In [ ]:
from campaña_sbf_modelo import campaña_sbf_modelo

# Leer el modelo INT8 .tflite en memoria
with open("googlenet_int8.tflite", "rb") as f:
    model_int8 = f.read()

# Usar solo los primeros 10 tensores para testeo
df_int8_reducido = df_int8.head(20)

# Ejecutar campaña SBF sobre pocos tensores INT8
df_resultados_int8 = campaña_sbf_modelo(
    model_content=model_int8,
    df_tensores=df_int8_reducido,
    x_input=x_test_ready,
    y_true=y_test,
    acc_baseline=acc_int8,
    tipo_modelo="int8",
    N=385
)

# Mostrar resultados
from IPython.display import display
print("🔍 Resultados INT8 (test reducido):")
display(df_resultados_int8)

# Guardar los resultados en csv
df_resultados_int8.to_csv("resultados_int8.csv", index=False)




🧬 Inyectando en: tfl.pseudo_qconst1 (index=3)


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


✅ Modelo: INT8  → 🎯 Accuracy: 0.5960

🧬 Inyectando en: tfl.pseudo_qconst3 (index=5)
✅ Modelo: INT8  → 🎯 Accuracy: 0.5960

🧬 Inyectando en: tfl.pseudo_qconst5 (index=7)
✅ Modelo: INT8  → 🎯 Accuracy: 0.5960

🧬 Inyectando en: tfl.pseudo_qconst7 (index=9)
✅ Modelo: INT8  → 🎯 Accuracy: 0.5970

🧬 Inyectando en: tfl.pseudo_qconst9 (index=11)
✅ Modelo: INT8  → 🎯 Accuracy: 0.5980

🧬 Inyectando en: tfl.pseudo_qconst11 (index=13)
✅ Modelo: INT8  → 🎯 Accuracy: 0.5970

🧬 Inyectando en: tfl.pseudo_qconst13 (index=15)
✅ Modelo: INT8  → 🎯 Accuracy: 0.5960

🧬 Inyectando en: tfl.pseudo_qconst15 (index=17)
✅ Modelo: INT8  → 🎯 Accuracy: 0.5980

🧬 Inyectando en: tfl.pseudo_qconst17 (index=19)
✅ Modelo: INT8  → 🎯 Accuracy: 0.5970

🧬 Inyectando en: tfl.pseudo_qconst19 (index=21)
✅ Modelo: INT8  → 🎯 Accuracy: 0.5970

🧬 Inyectando en: tfl.pseudo_qconst21 (index=23)
✅ Modelo: INT8  → 🎯 Accuracy: 0.5970

🧬 Inyectando en: tfl.pseudo_qconst23 (index=25)
✅ Modelo: INT8  → 🎯 Accuracy: 0.5970

🧬 Inyectando en: tfl.ps

,Index Modelo,Tensor,Forma,# Pesos,Total Bits Inyectables,Pesos Modificados,Acc Baseline,Acc Post-inyección,Degradación (%)
18,39,tfl.pseudo_qconst37,"(192, 1, 1, 1280)",245760,1966080,385,59.6,59.4,-0.2
0,3,tfl.pseudo_qconst1,"(1000, 2048)",2048000,16384000,385,59.6,59.6,0.0
2,7,tfl.pseudo_qconst5,"(384, 1, 3, 384)",442368,3538944,385,59.6,59.6,0.0
1,5,tfl.pseudo_qconst3,"(384, 3, 1, 384)",442368,3538944,385,59.6,59.6,0.0
14,31,tfl.pseudo_qconst29,"(384, 3, 1, 384)",442368,3538944,385,59.6,59.6,0.0
6,15,tfl.pseudo_qconst13,"(384, 1, 3, 384)",442368,3538944,385,59.6,59.6,0.0
17,37,tfl.pseudo_qconst35,"(320, 1, 1, 1280)",409600,3276800,384,59.6,59.6,0.0
3,9,tfl.pseudo_qconst7,"(384, 3, 3, 448)",1548288,12386304,385,59.6,59.7,0.1
11,25,tfl.pseudo_qconst23,"(384, 1, 3, 384)",442368,3538944,385,59.6,59.7,0.1
13,29,tfl.pseudo_qconst27,"(448, 1, 1, 1280)",573440,4587520,385,59.6,59.7,0.1


# FP32

In [ ]:
from campaña_sbf_modelo import campaña_sbf_modelo

# Leer el modelo .tflite en memoria
with open("googlenet_fp32.tflite", "rb") as f:
    model_fp32 = f.read()

# Usar solo los primeros 10 tensores para testeo
df_fp32_reducido = df_fp32.head(10)

# Ejecutar campaña SBF sobre pocos tensores para probar
df_resultados_fp32 = campaña_sbf_modelo(
    model_content=model_fp32,
    df_tensores=df_fp32_reducido,
    x_input=x_test_ready,
    y_true=y_test,
    acc_baseline=acc_fp32,
    tipo_modelo="fp32",
    N=385
)

# Mostrar resultados
from IPython.display import display
print("🔍 Resultados FP32 (test reducido):")
display(df_resultados_fp32)

# Guardar los resultados en csv
df_resultados_fp32.to_csv("resultados_fp32.csv", index=False)



🧬 Inyectando en: arith.constant (index=1)


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


✅ Modelo: FP32  → 🎯 Accuracy: 0.6090

🧬 Inyectando en: arith.constant1 (index=2)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant2 (index=3)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant3 (index=4)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0000

🧬 Inyectando en: arith.constant4 (index=5)
✅ Modelo: FP32  → 🎯 Accuracy: 0.6090

🧬 Inyectando en: arith.constant5 (index=6)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant6 (index=7)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant7 (index=8)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant8 (index=9)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant9 (index=10)
✅ Modelo: FP32  → 🎯 Accuracy: 0.0010
🔍 Resultados FP32 (test reducido):


,Index Modelo,Tensor,Forma,# Pesos,Total Bits Inyectables,Pesos Modificados,Acc Baseline,Acc Post-inyección,Degradación (%)
3,4,arith.constant3,"(448,)",448,14336,343,60.7,0.0,-60.7
1,2,arith.constant1,"(384,)",384,12288,339,60.7,0.1,-60.6
2,3,arith.constant2,"(384,)",384,12288,339,60.7,0.1,-60.6
5,6,arith.constant5,"(384,)",384,12288,340,60.7,0.1,-60.6
7,8,arith.constant7,"(320,)",320,10240,329,60.7,0.1,-60.6
6,7,arith.constant6,"(384,)",384,12288,340,60.7,0.1,-60.6
9,10,arith.constant9,"(384,)",384,12288,337,60.7,0.1,-60.6
8,9,arith.constant8,"(192,)",192,6144,304,60.7,0.1,-60.6
4,5,arith.constant4,"(384,)",384,12288,340,60.7,60.9,0.2
0,1,arith.constant,"(384,)",384,12288,352,60.7,60.9,0.2


# CH

In [ ]:
from campaña_sbf_modelo import campaña_sbf_modelo

# 📥 Invocación CH
with open("googlenet_ch.tflite", "rb") as f:
    model_ch = f.read()

df_ch_reducido = df_ch.head(5)

df_resultados_ch = campaña_sbf_modelo(
    model_content=model_ch,
    df_tensores=df_ch_reducido,
    x_input=x_test_ready,
    y_true=y_test,
    acc_baseline=acc_ch,
    tipo_modelo="ch",
    N=385
)

# 📊 Mostrar resultados
from IPython.display import display
print("🔍 CH:")
display(df_resultados_ch)

# Guardar los resultados en csv
df_resultados_ch.to_csv("resultados_ch.csv", index=False)



🧬 Inyectando en: arith.constant (index=1)


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


✅ Modelo: CH  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant1 (index=2)
✅ Modelo: CH  → 🎯 Accuracy: 0.0010

🧬 Inyectando en: arith.constant3 (index=4)
✅ Modelo: CH  → 🎯 Accuracy: 0.0000

🧬 Inyectando en: arith.constant4 (index=5)
✅ Modelo: CH  → 🎯 Accuracy: 0.6050

🧬 Inyectando en: arith.constant5 (index=6)
✅ Modelo: CH  → 🎯 Accuracy: 0.6040
🔍 CH:


,Index Modelo,Tensor,Forma,# Pesos,Total Bits Inyectables,Pesos Modificados,Acc Baseline,Acc Post-inyección,Degradación (%)
2,4,arith.constant3,"(1000,)",1000,32000,360,60.6,0.0,-60.6
0,1,arith.constant,"(32,)",32,1024,122,60.6,0.1,-60.5
1,2,arith.constant1,"(32, 3, 3, 3)",864,27648,363,60.6,0.1,-60.5
4,6,arith.constant5,"(384,)",384,12288,343,60.6,60.4,-0.2
3,5,arith.constant4,"(384,)",384,12288,336,60.6,60.5,-0.1
